## 1. Download a Sample Video

First, let's download a sample video and trim it to 10 seconds to make processing faster.

In [ ]:
# Install required packages
!pip install wget moviepy ultralytics opencv-python -q

In [ ]:
# Install required packages
!pip install wget moviepy ultralytics opencv-python -q

# Download a sample traffic video from Pexels
!wget "https://www.pexels.com/download/video/31653765/" -O original_race_video.mp4

# Check if the original video file exists and show its size
!ls -lh original_race_video.mp4

# Trim the first 10 seconds from the video to make processing faster
from moviepy.editor import VideoFileClip
print("Trimming  first 10 seconds from video...")

# Load the video and create a 10-second clip
original_clip = VideoFileClip("original_race_video.mp4")
trimmed_clip = original_clip
trimmed_clip = original_clip.subclip(10, )  # Remove first 10 seconds

# Save the trimmed video
trimmed_clip.write_videofile("race_video.mp4", codec="libx264", audio=False)
original_clip.close()
trimmed_clip.close()

# Check the size of the trimmed video
!ls -lh race_video.mp4
print("Video successfully trimmed by 10 seconds!")

## 2. Import Libraries and Set Up YOLOv8

Let's import the necessary libraries for our project.

In [ ]:
# Import required libraries
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from base64 import b64encode

print("Libraries imported successfully!")

## 3. Load a Pre-trained YOLOv8 Model

Now, let's download and load a pre-trained YOLOv8 model. We'll use the YOLOv8n model, which is relatively small and fast.

In [ ]:
# Load a pre-trained YOLOv8 model
model = YOLO("yolov8n.pt")

# Display model info
print(f"Model loaded: {model}")
print(f"Model task: {model.task}")

# Classes that YOLOv8 can detect
print("\nYOLOv8 can detect these classes:")
for i, class_name in enumerate(model.names.values()):
  if i<25:
    print(f"{i}: {class_name}")

Find the class IDs for horses (and animals that might be mistaken for horses) in the list above.
- 17: Horse
- 19: Cow

We'll focus on these classes for our computer vision task.

## 4. Detect Horses in One Frame

Let's extract a single frame from the video and detect horses in it.

In [ ]:
conf_threshold = 0.4

# Open the video file
cap = cv2.VideoCapture('race_video.mp4')

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Could not open video file")
else:
    # Read the first frame
    ret, frame = cap.read()

    if ret:
        # Get video details
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        print(f"Video details:")
        print(f"Resolution: {frame_width}x{frame_height}")
        print(f"FPS: {fps}")
        print(f"Total frames: {total_frames}")
        print(f"Duration: {total_frames/fps:.2f} seconds")

        # Save the first frame as an image
        cv2.imwrite('first_frame.jpg', frame)
        print("\nFirst frame saved as 'first_frame.jpg'")

        # Display the frame
        plt.figure(figsize=(10, 6))
        plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        plt.title("First frame of the (trimmed) video")
        plt.axis('off')
        plt.show()
    else:
        print("Error: Could not read the first frame")

    # Release the video capture object
    cap.release()

Now let's use YOLOv8 to detect horses in the first frame

In [ ]:
# Load the saved first frame
frame = cv2.imread('first_frame.jpg')

# Define the classes we're interested in (horse)
animal_classes = [17]  # COCO dataset class IDs for horse

# Run YOLOv8 inference on the frame
results = model(frame)

# Get detection results from the first detection (there's only one image)
result = results[0]

# Initialize a list to store horse detections
horse_detections = []

# Filter for vehicle classes and confidence threshold
for box in result.boxes:
    class_id = int(box.cls.item())
    confidence = box.conf.item()

    # Only include vehicles with confidence > conf_threshold
    if class_id in animal_classes and confidence > conf_threshold:
        # Get bounding box coordinates (convert from PyTorch tensor to integers)
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())

        # Add to our vehicle detections list
        class_name = model.names[class_id]
        horse_detections.append((x1, y1, x2, y2, confidence, class_name))

# Create a copy of the frame to draw on
annotated_frame = frame.copy()

# Draw bounding boxes for vehicle detections
for x1, y1, x2, y2, conf, class_name in horse_detections:
    # Draw rectangle
    cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Prepare label text
    label = f"{class_name}: {conf:.2f}"

    # Determine text size and position
    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    y1 = max(y1, text_height + 5)

    # Draw filled rectangle for text background
    cv2.rectangle(annotated_frame, (x1, y1 - text_height - 5), (x1 + text_width, y1), (0, 255, 0), -1)

    # Add text
    cv2.putText(annotated_frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

# Add a total count to the image
total_count = len(horse_detections)
count_text = f"RWWA - Intro to Data Science:  Total Horses: {total_count}"
cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

# Display the annotated frame
plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
plt.title("Horse Detection on First Frame")
plt.axis('off')
plt.show()

# Save the annotated frame
cv2.imwrite('annotated_first_frame.jpg', annotated_frame)
print(f"Detected {total_count} horses")
print("\nAnnotated frame saved as 'annotated_first_frame.jpg'")

## 5 & 6. Process the Entire Video and Show Results

Now, let's process the entire video, detect the horses in each frame, and create a new video with annotations.

In [ ]:
input_path = 'race_video.mp4'

In [ ]:
# Define a function to process the video
def process_video(input_path, output_path, model, target_classes, conf_threshold=0.5):
    # Open the video file
    cap = cv2.VideoCapture(input_path)

    # Check if the video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Create video writer for output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Initialize a counter for processed frames
    frame_count = 0

    # Process frames
    while cap.isOpened():
        # Read a frame
        success, frame = cap.read()

        if not success:
            break

        # Update frame count
        frame_count += 1

        # Print progress every 10 frames
        if frame_count % 10 == 0:
            print(f"Processing frame {frame_count}/{total_frames} ({frame_count/total_frames*100:.1f}%)")

        # Run YOLOv8 inference
        results = model(frame)
        result = results[0]

        # Filter for vehicle classes
        detections = []
        for box in result.boxes:
            class_id = int(box.cls.item())
            confidence = box.conf.item()

            if class_id in target_classes and confidence > conf_threshold:
                x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                class_name = model.names[class_id]
                detections.append((x1, y1, x2, y2, confidence, class_name))

        # Create a copy of the frame
        annotated_frame = frame.copy()

        # Draw bounding boxes
        for x1, y1, x2, y2, conf, class_name in detections:
            cv2.rectangle(annotated_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Prepare label text
            label = f"{class_name}: {conf:.2f}"

            # Determine text size and position
            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            y1 = max(y1, text_height + 5)

            # Draw filled rectangle for text background
            cv2.rectangle(annotated_frame, (x1, y1 - text_height - 5), (x1 + text_width, y1), (0, 255, 0), -1)

            # Add text
            cv2.putText(annotated_frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

        # Add a total count to the frame
        total_count = len(detections)
        count_text = f"RWWA - Intro to Data Science:  Total horses: {total_count}"
        cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Add frame number
        frame_text = f"Frame: {frame_count}/{total_frames}"
        cv2.putText(annotated_frame, frame_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Write the frame to the output video
        out.write(annotated_frame)

    # Release video capture and writer
    cap.release()
    out.release()

    print(f"\nProcessing complete! Output saved to {output_path}")
    return output_path

In [ ]:
# Define the target classes (horses)
vehicle_classes = [17]  # horse

# Process the video (this will be much faster with the 10-second clip)
output_path = process_video(
    input_path='race_video.mp4',
    output_path='annotated_race_video.mp4',
    model=model,
    target_classes=animal_classes,
    conf_threshold=conf_threshold
)

## Display the Annotated Video

Let's display the processed video with animal detections.

In [ ]:
!ffmpeg -y -i annotated_race_video.mp4 \
  -c:v libx264 -pix_fmt yuv420p -movflags +faststart \
  -crf 23 -preset veryfast \
  annotated_traffic_video_h264.mp4

In [ ]:
# Function to display the video in the notebook
def display_video(video_path):
    # Read the video file
    video_file = open(video_path, "rb")
    video_bytes = video_file.read()
    video_file.close()

    # Convert to base64
    encoded = b64encode(video_bytes).decode()

    # Display the video using HTML
    display(HTML(f"""
    <video width="800" height="600" controls>
        <source src="data:video/mp4;base64,{encoded}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """))

# Display the annotated video
print("Displaying the video with animal detections:")
display_video('annotated_race_video.mp4')

In [ ]:
!ffmpeg -y -i annotated_race_video.mp4 \
  -c:v libx264 -pix_fmt yuv420p -movflags +faststart \
  -crf 23 -preset veryfast \
  annotated_race_video_h264.mp4

In [ ]:
!zip -j annotated_race_video_h264.zip /content/annotated_race_video_h264.mp4
from google.colab import files
files.download("annotated_race_video_h264.zip")

## Conclusion

In this workshop, we've learned how to:
1. Download a sample video and trim it by 10 seconds for faster processing
2. Install YOLOv8 and necessary libraries
3. Load a pre-trained model
4. Detect horses in a single frame
5. Process the entire video
6. Display the results with labels and counts

## Resources and Further Reading

- [Ultralytics YOLOv8 Documentation](https://docs.ultralytics.com/)
- [OpenCV Documentation](https://docs.opencv.org/)
- [COCO Dataset Classes](https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/)
- [Object Tracking Tutorials](https://learnopencv.com/object-tracking-using-opencv-cpp-python/)